In [ ]:
%matplotlib inline

import iris
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from wildfires.data.datasets import GSMaP_dry_day_period, data_map_plot

In [ ]:
import warnings

warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")

In [ ]:
dry_day_period = GSMaP_dry_day_period()

In [ ]:
cube = dry_day_period.cubes[0]
print(cube)

In [ ]:
fig = data_map_plot(cube.collapsed("time", iris.analysis.MEAN))

In [ ]:
fig = data_map_plot(iris.analysis.maths.log(cube).collapsed("time", iris.analysis.MEAN))

In [ ]:
for i, single_time_cube in enumerate(cube.slices_over("time")):
    if i % 30 == 0:
        print(single_time_cube.coord("time").cell(0).point)
        plt.figure()
        plt.hist(single_time_cube.data.data.flatten())
        plt.yscale("log")
        plt.xscale("log")
        plt.show()